In [1]:
import tensorflow as tf
import os
from PIL import Image

In [2]:
image_dir = 'c:/tmp/Temp_data_Set/Test_Dataset_png/'
image_list = os.listdir(image_dir)
image_list.sort()    # sort()는 return이 없음

for i in range(len(image_list)):
    image_list[i] = image_dir + image_list[i]
    if i%1000 == 0:
        print(image_list[i])

label_dir = 'c:/tmp/Temp_data_Set/Test_Dataset_csv/label.csv'
label_list = [label_dir]
print(label_list)

imagename_queue = tf.train.string_input_producer(image_list)
labelname_queue = tf.train.string_input_producer(label_list)

c:/tmp/Temp_data_Set/Test_Dataset_png/Face00001.png
c:/tmp/Temp_data_Set/Test_Dataset_png/Face01001.png
c:/tmp/Temp_data_Set/Test_Dataset_png/Face02001.png
c:/tmp/Temp_data_Set/Test_Dataset_png/Face03001.png
c:/tmp/Temp_data_Set/Test_Dataset_png/Face04001.png
c:/tmp/Temp_data_Set/Test_Dataset_png/Face05001.png
c:/tmp/Temp_data_Set/Test_Dataset_png/Face06001.png
c:/tmp/Temp_data_Set/Test_Dataset_png/Face07001.png
['c:/tmp/Temp_data_Set/Test_Dataset_csv/label.csv']


In [3]:
image_height = 61
image_width = 49

shuffle_batch_size = 32

label_class_size = 100

In [4]:
image_reader = tf.WholeFileReader()
label_reader = tf.TextLineReader()
image_key, image_value = image_reader.read(imagename_queue)
label_key, label_value = label_reader.read(labelname_queue)

In [5]:
image_decoded = tf.cast(tf.image.decode_png(image_value), tf.float32)
label_decoded = tf.cast(tf.decode_csv(label_value, record_defaults=[[0]]), tf.float32)
#print(image)

label = tf.reshape(label_decoded,[1])
image = tf.reshape(image_decoded,[image_height, image_width, 1])

x, y_true_cls_batch = tf.train.shuffle_batch(tensors=[image, label], batch_size=shuffle_batch_size, num_threads=4, capacity=5000, min_after_dequeue=100)

In [7]:
# label class 값을 one-hot encoding값으로 변환
# 0 <= label < label_class_size
# http://blogs.candoerz.com/question/122513/tensorflow-one-hot-encoder.aspx
# http://stackoverflow.com/questions/33681517/tensorflow-one-hot-encoder
sparse_labels = tf.reshape(tf.cast(y_true_cls_batch, tf.int32), [-1, 1])
derived_size = tf.shape(sparse_labels)[0]
indices = tf.reshape(tf.range(0, derived_size, 1), [-1, 1])
concated = tf.concat([indices, sparse_labels], 1)    # tf1.0
outshape = tf.concat([tf.reshape(derived_size, [1]), tf.reshape(label_class_size, [1])], 0)    # tf1.0
y_true = tf.sparse_to_dense(concated, outshape, 1.0, 0.0)
# one-hot encoding값을 readable 값으로 바꿈 : y_true_cls_batch의 shape를 조정하는 대신 y_pred_cls와 동일하게 처리
y_true_cls = tf.argmax(y_true, dimension=1)

In [8]:
W_conv1 = tf.Variable(tf.truncated_normal([5,5,1,32]))
b_conv1 = tf.Variable(tf.zeros([32]))

W_conv2 = tf.Variable(tf.truncated_normal([5,5,32,64]))
b_conv2 = tf.Variable(tf.zeros([64]))

W_fc1 = tf.Variable(tf.truncated_normal([image_width*image_height*64, 50]))
b_fc1 = tf.Variable(tf.zeros([50]))

W_fc2 = tf.Variable(tf.truncated_normal([50, 100]))
b_fc2 = tf.Variable(tf.zeros([100]))

In [9]:
x_image = tf.reshape(x, [-1, image_width, image_height, 1])

keep_prob = tf.placeholder(tf.float32)

In [10]:
h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], padding="SAME") + b_conv1)
h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1], strides=[1, 1, 1, 1], padding="SAME")

h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], padding="SAME") + b_conv2)
h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1], strides=[1, 1, 1, 1], padding="SAME")
h_pool2_flat = tf.reshape(h_pool2, [-1, image_width*image_height*64])

h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [11]:
print(y_conv)
print(y_true)

Tensor("add_3:0", shape=(32, 100), dtype=float32)
Tensor("SparseToDense_1:0", shape=(?, ?), dtype=float32)


In [12]:
#y_true = tf.placeholder(tf.float32, [None, label_class_size])
#y_true_cls = tf.placeholder(tf.int64, [None])

y_pred = tf.nn.softmax(logits=y_conv)
# one-hot encoding값을 readable 값으로 바꿈
y_pred_cls = tf.argmax(y_pred, dimension=1)

In [13]:
loss = tf.reduce_min(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))
train = tf.train.AdamOptimizer(1e-4).minimize(loss)

In [14]:
# prediction한 값과 label값을 비교 : True/False 리스트
correct_prediction = tf.equal(y_pred_cls, y_true_cls)

# True/False 리스트를 실수형으로 casting한다.
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [15]:
with tf.Session() as sess:
    print("started")
    coord = tf.train.Coordinator()
    thread = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    sess.run(tf.global_variables_initializer())

    for i in range(100):
#        print(sess.run(image_key))
#        print(sess.run(image_key))
        sess.run(train, {keep_prob: 0.7})
#        sess.run(train)

        if i % 50 == 0:
            #if train==True:
            _loss = sess.run(loss, {keep_prob: 0.7})
            _accuracy = sess.run(accuracy, {keep_prob: 0.7})
            #if release_mode==Train:
            #result = sess.run(pred, {keep_prob: 1.0})

            print("-----------------")
            print("loss: ", _loss)
            print("accuracy: ", _accuracy)

#            print(sess.run(image_key))
            print(sess.run(y_pred_cls, {keep_prob: 0.7}))
            print(sess.run(y_true_cls, {keep_prob: 0.7}))

#    print(image)

#    Image.fromarray(image).show()

    print("training done")
    
    coord.request_stop()
    coord.join(thread)

started
-----------------
loss:  4.62221
accuracy:  0.0
b'c:/tmp/Temp_data_Set/Test_Dataset_png/Face03378.png'
[77 29  0  3 39 39 39  3 39 39 86 39 39 57 39 39 39 99  3 39 99  0 39  3 39
 99  3 39 39 39 57 57]
[28 28 28 10 51 51 28 28 28 51 28 28 28 51 75 28 28 51 28 28 28 28 28  5 28
 28 28 51 16 51 28 51]
-----------------
loss:  4.62221
accuracy:  0.0
b'c:/tmp/Temp_data_Set/Test_Dataset_png/Face00196.png'
[57 31 39 57 39 39 57 39 99 39 39  3 86 39 44  3 39 39 44 99 39 39  3 39 39
  3 44 39 39 57  6 39]
[51 28 28 28 28 28 28 28 28 51 51  5 28 28 51 28 28 28 28 75 28 28 51 28 28
 28 51 28 51 51 28 28]
training done
